<a href="https://colab.research.google.com/github/daria-lpch/study_cnn_eeg/blob/main/StudyCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>